In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('dataset/train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
df = df.dropna()

In [6]:
X = df.drop('label',axis=1)

In [7]:
y = df['label']

In [8]:
y.shape

(18285,)

In [9]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [15]:
messages = X.copy()

In [16]:
messages.reset_index(inplace=True)

In [17]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [18]:
corpus=[]
for i in range(0,len(messages)):
    review = re.sub('[^a-zA-Z]',' ',messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [19]:
corpus 

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [20]:
voc_size = 5000

In [21]:
onehot_rep = [one_hot(words,voc_size)for words in corpus]

In [67]:
import pickle
pickle.dump(onehot_rep,open("../model/onehot_model.pkl","wb"))

In [68]:
import pickle
with open("onehot_rep","wb") as f:
    pickle.dump(onehot_rep, f)

In [22]:
onehot_rep

[[4233, 4082, 1698, 3903, 1671, 3385, 2569, 3779, 3946, 1991],
 [1784, 3392, 2920, 2703, 1001, 4802, 4096],
 [1209, 4335, 1091, 2322],
 [1724, 4853, 1110, 4719, 4535, 1783],
 [2379, 1001, 2086, 4791, 2317, 2812, 1001, 2124, 4737, 852],
 [467,
  2674,
  329,
  3172,
  530,
  4221,
  3747,
  703,
  2344,
  3201,
  4386,
  2872,
  3088,
  3663,
  4096],
 [691, 3434, 4118, 4467, 182, 1926, 3718, 4316, 3409, 1173, 2939],
 [3983, 3321, 2988, 4345, 4464, 1528, 4221, 3266, 3409, 1173, 2939],
 [4162, 975, 20, 2537, 1677, 1221, 2050, 848, 4221, 4641],
 [4206, 4706, 213, 445, 3836, 2499, 1771, 96],
 [2601, 3477, 1695, 841, 4977, 1021, 4797, 2497, 2686, 2587, 3647],
 [4719, 1576, 1671, 1221, 4221, 4464],
 [866, 3964, 1032, 1798, 1783, 3360, 4109, 3525, 3280],
 [1685, 3297, 2682, 1843, 248, 468, 654, 3409, 1173, 2939],
 [4986, 2909, 21, 2214, 3318, 3409, 1173, 2939],
 [1039, 2756, 3379, 4533, 754, 2699, 4081, 1443, 1860, 4507],
 [1739, 2323, 3392],
 [283, 3278, 1578, 2455, 4221, 890, 2936, 4096],
 

In [23]:
sen_length = 20
embedded_docs = pad_sequences(onehot_rep,padding='pre',maxlen=sen_length) # pre assign zero in the front to get  same length
embedded_docs

array([[   0,    0,    0, ..., 3779, 3946, 1991],
       [   0,    0,    0, ..., 1001, 4802, 4096],
       [   0,    0,    0, ..., 4335, 1091, 2322],
       ...,
       [   0,    0,    0, ..., 3409, 1173, 2939],
       [   0,    0,    0, ...,  933, 2491, 2943],
       [   0,    0,    0, ..., 1232, 1378, 4828]])

In [24]:
len(embedded_docs)

18285

In [26]:
#creating model
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sen_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [27]:
import numpy as np
x_final = np.array(embedded_docs)
y_final= np.array(y)

In [29]:
y_final.shape, x_final.shape

((18285,), (18285, 20))

In [31]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(x_final,y_final,test_size=0.33,random_state=42)

In [32]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Train on 12250 samples, validate on 6035 samples
Epoch 1/10
12250/12250 [==============================] - 13s 1ms/sample - loss: 0.3400 - accuracy: 0.8360 - val_loss: 0.1995 - val_accuracy: 0.9138
Epoch 2/10
12250/12250 [==============================] - 6s 498us/sample - loss: 0.1357 - accuracy: 0.9473 - val_loss: 0.2046 - val_accuracy: 0.9176
Epoch 3/10
12250/12250 [==============================] - 6s 486us/sample - loss: 0.0928 - accuracy: 0.9657 - val_loss: 0.2169 - val_accuracy: 0.9147
Epoch 4/10
12250/12250 [==============================] - 6s 529us/sample - loss: 0.0623 - accuracy: 0.9785 - val_loss: 0.2708 - val_accuracy: 0.9143
Epoch 5/10
12250/12250 [==============================] - 6s 523us/sample - loss: 0.0417 - accuracy: 0.9857 - val_loss: 0.3314 - val_accuracy: 0.9171
Epoch 6/10
12250/12250 [==============================] - 7s 599us/sample - loss: 0.0236 - accuracy: 0.9927 - val_loss: 0.3475 - val_accuracy: 0.9140
Epoch 7/10
12250/12250 [============================

In [35]:
y_predict =model.predict_classes(X_test)

In [36]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_predict)

array([[3087,  332],
       [ 207, 2409]], dtype=int64)

In [38]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_predict)

0.9106876553438277

In [62]:
x = ['civilian kill singl us airstrik identifi']


In [63]:
onehot_X = [one_hot(words,voc_size)for words in x]

In [64]:
x= pad_sequences(onehot_X,padding='pre',maxlen=sen_length)

In [65]:
result=model.predict_classes(x)

In [66]:
result[0][0]

1

In [61]:
model.save('lstm_fakenews_model.h5')